# BERT model pre-training from scratch

In [89]:
import math
import re
from   random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import spacy

torch.set_default_tensor_type('torch.cuda.FloatTensor') # for gpu training

### Import small subset of Shakespeare's work

In [90]:
with open ("input.txt", "r") as f:
    raw_text = f.read()
nlp = spacy.load("en_core_web_sm")
doc = nlp(raw_text)
sentences = list(doc.sents)

text = [x.text.lower() for x in sentences] #lower case
text = [re.sub("[.,!?\\-]", '', x) for x in text] #clean all symbols

text

['first citizen:\nbefore we proceed any further hear me speak\n\n',
 'all:\nspeak speak\n\n',
 'first citizen:\nyou are all resolved rather to die than to famish\n\n',
 'all:\nresolved resolved\n\n',
 'first citizen:\nfirst you know caius marcius is chief enemy to the people\n\n',
 "all:\nwe know't we know't\n\n",
 "first citizen:\nlet us kill him and we'll have corn at our own price\nis't a verdict\n\n",
 "all:\nno more talking on't; let it be done: away away\n\nsecond citizen:\none word good citizens\n\n",
 'first citizen:\nwe are accounted poor citizens the patricians good\n',
 'what authority surfeits on would relieve us: if they\nwould yield us but the superfluity while it were\nwholesome we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us the object of our misery is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them let us revenge this with\nour pikes ere we become rakes: for the gods know i\n

In [91]:
for sentence in text:
    print(sentence, "_____")
    words = sentence.split()
    print(words)

first citizen:
before we proceed any further hear me speak

 _____
['first', 'citizen:', 'before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak']
all:
speak speak

 _____
['all:', 'speak', 'speak']
first citizen:
you are all resolved rather to die than to famish

 _____
['first', 'citizen:', 'you', 'are', 'all', 'resolved', 'rather', 'to', 'die', 'than', 'to', 'famish']
all:
resolved resolved

 _____
['all:', 'resolved', 'resolved']
first citizen:
first you know caius marcius is chief enemy to the people

 _____
['first', 'citizen:', 'first', 'you', 'know', 'caius', 'marcius', 'is', 'chief', 'enemy', 'to', 'the', 'people']
all:
we know't we know't

 _____
['all:', 'we', "know't", 'we', "know't"]
first citizen:
let us kill him and we'll have corn at our own price
is't a verdict

 _____
['first', 'citizen:', 'let', 'us', 'kill', 'him', 'and', "we'll", 'have', 'corn', 'at', 'our', 'own', 'price', "is't", 'a', 'verdict']
all:
no more talking on't; let it be done: away away

seco

### Getting the vocabulary

In [92]:
# combine everything into one to make vocabs
word_list = list(set(" ".join(text).split()))
word2id = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}  #special tokens.

#create the word2id
for i, w in enumerate(word_list):
    word2id[w] = i + 4  #because 0-3 are already occupied
    id2word = {i: w for i, w in enumerate(word2id)}
    vocab_size = len(word2id)

#list of all tokens for whole text
token_list = list()
for sentence in sentences:
    arr = [word2id[word] for sentence in text for word in sentence.split()]
    token_list.append(arr)

In [95]:
#testing one sentence
for tokens in token_list[0]:
    print(id2word[tokens])

first
citizen:
before
we
proceed
any
further
hear
me
speak
all:
speak
speak
first
citizen:
you
are
all
resolved
rather
to
die
than
to
famish
all:
resolved
resolved
first
citizen:
first
you
know
caius
marcius
is
chief
enemy
to
the
people
all:
we
know't
we
know't
first
citizen:
let
us
kill
him
and
we'll
have
corn
at
our
own
price
is't
a
verdict
all:
no
more
talking
on't;
let
it
be
done:
away
away
second
citizen:
one
word
good
citizens
first
citizen:
we
are
accounted
poor
citizens
the
patricians
good
what
authority
surfeits
on
would
relieve
us:
if
they
would
yield
us
but
the
superfluity
while
it
were
wholesome
we
might
guess
they
relieved
us
humanely;
but
they
think
we
are
too
dear:
the
leanness
that
afflicts
us
the
object
of
our
misery
is
as
an
inventory
to
particularise
their
abundance;
our
sufferance
is
a
gain
to
them
let
us
revenge
this
with
our
pikes
ere
we
become
rakes:
for
the
gods
know
i
speak
this
in
hunger
for
bread
not
in
thirst
for
revenge


### Dataloaders

In [96]:
batch_size = 6
max_mask   = 5  # max masked tokens when 15% exceed, it will only be max_pred
max_len    = 1000 # maximum of length to be padded; 

In [97]:
def make_batch():
    batch = []
    positive = negative = 0  #count of batch size;  we want to have half batch that are positive pairs (i.e., next sentence pairs)
    while positive != batch_size/2 or negative != batch_size/2:
        
        #randomly choose two sentence so we can put [SEP]
        tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences))
        #retrieve the two sentences
        tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]

        #1. token embedding - append CLS and SEP
        input_ids = [word2id['[CLS]']] + tokens_a + [word2id['[SEP]']] + tokens_b + [word2id['[SEP]']]

        #2. segment embedding - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        #3. mask language modeling
        #masked 15%, but should be at least 1 but does not exceed max_mask
        n_pred =  min(max_mask, max(1, int(round(len(input_ids) * 0.15))))
        #get the pos that excludes CLS and SEP and shuffle them
        cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] and token != word2id['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        #simply loop and change the input_ids to [MASK]
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)  #remember the position
            masked_tokens.append(input_ids[pos]) #remember the tokens
            #80% replace with a [MASK], but 10% will replace with a random token
            if random() < 0.1:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word2id[id2word[index]] # replace
            elif random() < 0.9:  # 80%
                input_ids[pos] = word2id['[MASK]'] # make mask
            else:  #10% do nothing
                pass

        # pad the input_ids and segment ids until the max len
        n_pad = max_len - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # pad the masked_tokens and masked_pos to make sure the lenth is max_mask
        if max_mask > n_pred:
            n_pad = max_mask - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        #check if first sentence is really comes before the second sentence
        #also make sure positive is exactly half the batch size
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
            
    return batch

batch = make_batch()

len(batch)

In [100]:
# Deconstruct using map and zip
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
input_ids.shape, segment_ids.shape, masked_tokens.shape, masked_pos.shape, isNext.shape
input_ids, segment_ids, masked_tokens, masked_pos, isNext = input_ids.to(device), segment_ids.to(device), masked_tokens.to(device), masked_pos.to(device), isNext.to(device)


## BERT Model definition

### Embedding

In [101]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_len, d_model)      # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        #x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

### Attention mask

In [102]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

### Encoder

In [104]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn       = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

In [105]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn 

In [106]:
n_layers = 6    # number of Encoder of Encoder Layer
n_heads  = 8    # number of heads in Multi-Head Attention
d_model  = 768  # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

In [107]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x len_q x d_model]


In [108]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(F.gelu(self.fc1(x)))


## BERT Model

In [109]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        
        # 1. predict next sentence
        # it will be decided by first token(CLS)
        h_pooled   = self.activ(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_nsp = self.classifier(h_pooled) # [batch_size, 2]

        # 2. predict the masked token
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked  = self.norm(F.gelu(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_nsp
    

    

## Training loop

In [110]:
num_epoch = 100
model = BERT()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
input_ids, segment_ids, masked_tokens, masked_pos, isNext = input_ids.to(device), segment_ids.to(device), masked_tokens.to(device), masked_pos.to(device), isNext.to(device)
for epoch in range(num_epoch):
    optimizer.zero_grad()
    logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)    

    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()

    loss_nsp = criterion(logits_nsp, isNext) # for sentence classification
    
    loss = loss_lm + loss_nsp
    if epoch % 10 == 0:
        print('Epoch:', '%02d' % (epoch), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 00 loss = 77.911346
Epoch: 10 loss = 92.424904
Epoch: 20 loss = 76.264832
Epoch: 30 loss = 42.310841
Epoch: 40 loss = 23.345474
Epoch: 50 loss = 19.225452
Epoch: 60 loss = 17.568678
Epoch: 70 loss = 10.960737
Epoch: 80 loss = 8.401817
Epoch: 90 loss = 6.422518


## Inference

In [117]:
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[2]))
print([id2word[w.item()] for w in input_ids[0] if id2word[w.item()] != '[PAD]'])
input_ids, segment_ids, masked_tokens, masked_pos, isNext = input_ids.to(device), segment_ids.to(device), masked_tokens.to(device), masked_pos.to(device), isNext.to(device)

logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)

logits_lm = logits_lm.data.max(2)[1][0].data

print('masked tokens (words) : ',[id2word[pos.item()] for pos in masked_tokens[0]])
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0]])
print('masked tokens (words) : ',[id2word[pos.item()] for pos in logits_lm])
print('predict masked tokens list : ', [pos for pos in logits_lm])

logits_nsp = logits_nsp.data.max(1)[1][0].data
print(logits_nsp)
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_nsp else False)

['[CLS]', 'first', 'citizen:', 'before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', 'all:', 'speak', 'speak', 'first', 'citizen:', 'you', 'are', 'all', 'resolved', 'rather', 'to', 'die', 'than', 'to', 'famish', 'all:', 'resolved', 'resolved', 'first', 'citizen:', 'first', 'you', '[MASK]', 'caius', 'marcius', 'is', 'chief', 'enemy', 'to', 'the', 'people', 'all:', 'we', "know't", 'we', "know't", 'first', 'citizen:', 'let', 'us', 'kill', 'him', 'and', "we'll", 'have', 'corn', 'at', 'our', 'own', 'price', "is't", 'a', 'verdict', 'all:', 'no', 'more', 'talking', "on't;", 'let', 'it', 'be', 'done:', 'away', 'away', 'second', 'citizen:', 'one', 'word', 'good', 'citizens', 'first', 'citizen:', 'we', 'are', 'accounted', 'poor', 'citizens', 'the', 'patricians', 'good', 'what', 'authority', 'surfeits', 'on', 'would', 'relieve', 'us:', 'if', 'they', 'would', 'yield', 'us', 'but', 'the', 'superfluity', 'while', 'it', 'were', 'wholesome', 'we', 'might', 'guess', 'they', 'relieved', 'u

### This notebook is only for demonstration of pre-training BERT. The actual BERT model is much larger and also uses lot of training data: Toronto BookCorpus (800M words) and English Wikipedia (2,500M words). Training this requires sufficient compute and not possible on a single local GPU machine.  

### After the model is pre-trained, it can be fine-tuned for downstream tasks such as sequence classficiation, Natural language inference etc. 